In [2]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns",999)

In [3]:
def all_values(df):
    for i in df.columns:
        print("")
        print(i,":", df[i].value_counts(),"\n")

### Ideas
* Instead of horoscope sign, just do "seriousness" of horoscopes
  * DONE
* Religion column and "faith intensity" column? "" with horoscope
  * DONE
* Diet can be bifurcated ('stricts', 'mostlys', everything else)
  * DONE
* For SPEAKS - maybe English == English (fluent) Y/N, and then anything else fluent for a few languages?
  * DONE
* Word count, or verb count, for all essay columns
* Should income be categorical or continuous?
* Should I put NaNs in for all incomes of -1 (did not answer?). It's 48000+ observations...
* Education: I think I want to group with highest level of achievement AND make groupings for current vs non-current student

### Simple, basic transformations
* **Drop** essays, last_online, drugs, drinks, diet, religion, speaks, location, ethnicity, offspring, sign
*

# Helper Functions

In [4]:
def number_of_uniques(df):
    for i in df.columns:
        print(i,":", len(df[i].unique()))
        
def number_of_NaN(df):
    for i in df.columns:
        print(i,":", df[i].isna().sum())

# Mandatory Cleaning

In [5]:
cupid = pd.read_csv('okcupid/profiles.csv')

### Once ready for modeling, I should certainly drop:

# cupid.drop(columns=['last_online','location'

In [6]:
# Creating new columns
cupid['diet_intensity'] = cupid.diet.str.extract('(\w+ )')
cupid.loc[ ( cupid.diet_intensity.isna() ) & ( cupid.diet.notna() ), 'diet_intensity'] = "unanswered"
cupid['diet_choice'] = cupid.diet.str.split(" ").str[-1]
cupid['primary_ethnicity'] = cupid.ethnicity.str.replace(", ",",").str.split(",",expand=True)[0]

cupid['wants_kids'] = cupid.offspring.str.extract('(\S*\w*\swant.*)', expand=True)[0]
cupid['wants_kids'] = cupid.wants_kids.str.replace(".*might.*", "maybe",regex=True)
cupid['wants_kids'] = cupid.wants_kids.str.replace(".*does.*", "no",regex=True)
cupid['wants_kids'] = cupid.wants_kids.str.replace(".*wants.*", "yes",regex=True)
cupid.loc[ ( cupid.wants_kids.isna() ) & ( cupid.offspring.notna() ), 'wants_kids'] = "unanswered"


cupid['has_kids'] = cupid.offspring.str.extract('((\S+\s|^)(have|has)(\s+\S+|$))', expand=True)[0]
cupid['has_kids'] = cupid.has_kids.str.replace(".*does.*", "no",regex=True)
cupid['has_kids'] = cupid.has_kids.str.replace(".*has a.*", "one",regex=True)
cupid['has_kids'] = cupid.has_kids.str.replace(".*has kids.*", "multiple",regex=True)
cupid.loc[ ( cupid.has_kids.isna() ) & ( cupid.offspring.notna() ), 'has_kids'] = "unanswered"


cupid['likes_cats'] = cupid.pets.str.contains("(^|\s+)likes cats", regex=True)
cupid['likes_dogs'] = cupid.pets.str.contains("(^|\s+)likes dogs", regex=True)
cupid['dislikes_cats'] = cupid.pets.str.contains("dislikes cats")
cupid['dislikes_dogs'] = cupid.pets.str.contains("dislikes dogs")
cupid['has_cats'] = cupid.pets.str.contains("has cats")
cupid['has_dogs'] = cupid.pets.str.contains("has dogs")

cupid['english_fluent'] = cupid.speaks.str.strip().str.extract("(english($|,|\s\(f))")[0]
cupid['english_fluent'] = cupid.english_fluent.str.replace('.*english.*', "english", regex=True)
cupid.loc[ ( cupid.english_fluent.isna() ) & ( cupid.speaks.notna() ), 'english_fluent'] = "placeholder"

cupid['english_poor'] = cupid.speaks.str.extract("(english \([po])")[0]
cupid['english_poor'] = cupid.english_poor.str.replace('.*english.*', "poor", regex=True)
cupid.loc[ ( cupid.english_poor.isna() ) & ( cupid.speaks.notna() ), 'english_poor'] = "placeholder"

cupid['spanish_fluent'] =cupid.speaks.str.contains("spanish \(f", regex=True)
cupid.loc[ ( cupid.spanish_fluent.isna() ) & ( cupid.speaks.notna() ), 'spanish_fluent'] = "placeholder"

cupid['spanish_not_poorly'] = cupid.speaks.str.contains("spanish \([po]",regex=True)
cupid.loc[ ( cupid.spanish_not_poorly.isna() ) & ( cupid.speaks.notna() ), 'spanish_not_poorly'] = "placeholder"

cupid['signs_fun'] = cupid.sign.str.extract("(fun to think about)|(but it)|(matters a lot)")[0]
cupid.loc[ ( cupid.signs_fun.isna() ) & ( cupid.sign.notna() ), 'signs_fun'] = "placeholder"

cupid['signs_unimportant'] = cupid.sign.str.extract("(fun to think about)|(but it)|(matters a lot)")[1]
cupid.loc[ ( cupid.signs_unimportant.isna() ) & ( cupid.sign.notna() ), 'signs_unimportant'] = "placeholder"

cupid['signs_important'] = cupid.sign.str.extract("(fun to think about)|(but it)|(matters a lot)")[2]
cupid.loc[ ( cupid.signs_important.isna() ) & ( cupid.sign.notna() ), 'signs_important'] = "placeholder"

cupid['religion_unserious'] = cupid.religion.str.extract("(not too serious)|(laughing)|(somewhat)|(very serious)")[0]
cupid.loc[ ( cupid.religion_unserious.isna() ) & ( cupid.religion.notna() ), 'religion_unserious'] = "placeholder"

cupid['religion_laughing'] =cupid.religion.str.extract("(not too serious)|(laughing)|(somewhat)|(very serious)")[1]
cupid.loc[ ( cupid.religion_laughing.isna() ) & ( cupid.religion.notna() ), 'religion_laughing'] = "placeholder"

cupid['religion_somewhat'] = cupid.religion.str.extract("(not too serious)|(laughing)|(somewhat)|(very serious)")[2]
cupid.loc[ ( cupid.religion_somewhat.isna() ) & ( cupid.religion.notna() ), 'religion_somewhat'] = "placeholder"

cupid['religion_serious'] =cupid.religion.str.extract("(not too serious)|(laughing)|(somewhat)|(very serious)")[3]
cupid.loc[ ( cupid.religion_serious.isna() ) & ( cupid.religion.notna() ), 'religion_serious'] = "placeholder"

cupid['religion_name'] = cupid.religion.str.extract("(\w+)")
#### education changes
cupid['new_education'] = cupid['education'].copy(deep=True)

cupid.loc[cupid.new_education == "high school", 'new_education'] = "graduated from high school"
cupid.loc[cupid.new_education == "law school", 'new_education'] = "graduated from law school"
cupid.loc[cupid.new_education == "masters program", 'new_education'] = "graduated from masters program"
cupid.loc[cupid.new_education == "med school", 'new_education'] = "graduated from med school"
cupid.loc[cupid.new_education == "ph.d program", 'new_education'] = "graduated from ph.d program"
cupid.loc[cupid.new_education == "space camp", 'new_education'] = "graduated from space camp"
cupid.loc[cupid.new_education == "two-year college", 'new_education'] = "graduated from two-year college"
cupid.loc[cupid.new_education == "college/university", 'new_education'] = "graduated from college/university"
#### maybe I'll use these
# cupid['education_droppedout'] = cupid.new_education.str.startswith("dropped out")
# cupid['education_graduated'] = cupid.new_education.str.startswith("graduated from")
# cupid['education_currentstudent'] = cupid.new_education.str.startswith("working on")

### create binary_smokes
cupid['binary_smokes'] = cupid['smokes'].copy()
cupid['binary_smokes'] = cupid.smokes.str.replace("(sometimes)|(when drinking)|(yes)|(trying to quit)", "yes",regex=True)

/home/spencer/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
/home/spencer/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [45]:
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

cupid.to_hdf('data/cupid_cleaned.hd5', key='df', mode='w')

# import pickle
# with open('data/cupid_cleaned.pkl', 'wb') as picklefile:
#     pickle.dump(cupid, picklefile)

In [29]:
# print('smokes:',cupid.smokes.unique())
# print('drinks:',cupid.drinks.unique())
# print('drugs:',cupid.drugs.unique())
print('last_online:',cupid.last_online.unique())

last_online: ['2012-06-28-20-30' '2012-06-29-21-41' '2012-06-27-09-10' ...
 '2012-06-02-08-16' '2012-02-17-20-44' '2012-06-14-16-51']


# Optional Cleaning

In [16]:
cupid = cupid.drop(['essay0', 'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6', 'essay7', 'essay8', 'essay9',
                    'last_online', 'drinks', 'drugs', 'diet', 'religion', 'speaks', 'location', 'ethnicity', 'offspring', 'sign'], axis=1)

num